In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Scraping a website.

In [2]:
import requests

def get_text(url):
      headers = {
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
        }
      return requests.get(url, headers=headers).text

In [16]:
jina = 'https://r.jina.ai/'
my_url = input('Enter a url ')
text_1 = get_text(url=jina + my_url)
text_1

Enter a url https://deerfield.com/


'Title: Deerfield\n\nURL Source: https://deerfield.com/\n\nMarkdown Content:\n[![Image 1: Deerfield](https://deerfield.com/wp-content/themes/deerfieldwptheme/assets/img/Deerfield_30th-Anniversary_Logo_Black.png)](https://deerfield.com/)\n\n![Image 2: About Deerfield](https://deerfield.com/wp-content/uploads/2022/05/cure_lobby_full.jpeg)\n\nLaunched in 1994, Deerfield Management Company is an investment firm dedicated to advancing healthcare through information, investment, and philanthropy—all toward the end goal of cures for disease, improved quality of life, and reduced cost of care.\n\nRead More\n\n![Image 3: ---](https://deerfield.com/wp-content/uploads/2022/05/Investment-scaled.jpeg)\n\n##### Investment\n\nSupporting companies across the healthcare ecosystem with flexible funding models…\n\nRead More\n\n![Image 4: ---](https://deerfield.com/wp-content/uploads/2022/05/Film_screenshot.jpeg)\n\n##### Information\n\nDelivering market research to the Deerfield team, its portfolio compa

In [17]:
len(text_1)

4474

# Building a database.

In [5]:
database = ['https://greylock.com/', 'https://a16z.com/', 'https://www.sequoiacap.com/', 'https://www.indexventures.com/',
            'https://www.kleinerperkins.com/', 'https://lsvp.com/', 'https://matrix.vc/', 'https://www.500.co/',
            'https://www.sparkcapital.com/']

import pickle

# with open ('database', 'rb') as fp:   # if exists
#    database = pickle.load(fp)

In [18]:
texts = []
for x in database:
      text = get_text(url=jina + x)
      texts.append(text)
len(texts)

10

# Building our question answering model.

In [19]:
from transformers import pipeline

In [20]:
my_questions = ["What is the name of this Venture Capital firm?", "Who are the contacts of this company?",
                "In which type of industries this firm invests in?", "In which investment rounds this firm participates or leads?"]

In [21]:
# download model
qa_model = pipeline("question-answering")

# test model
for i in my_questions:
      qa_response = qa_model(question = i, context = text_1)
      print( '\n ', i, qa_response)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.



  What is the name of this Venture Capital firm? {'score': 0.702899158000946, 'start': 350, 'end': 378, 'answer': 'Deerfield Management Company'}

  Who are the contacts of this company? {'score': 0.04564114287495613, 'start': 939, 'end': 997, 'answer': 'Deerfield team, its portfolio companies and other partners'}

  In which type of industries this firm invests in? {'score': 0.4169709384441376, 'start': 1585, 'end': 1660, 'answer': 'life science, medical device, diagnostic, digital health and health service'}

  In which investment rounds this firm participates or leads? {'score': 0.17612609267234802, 'start': 4276, 'end': 4326, 'answer': 'Select a category or search to explore the network'}


# Building a similary analysis model.

In [22]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

d = {}
# Load pre-trained BERT model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

for i in range(len(texts)):
# Sample sentences
     sentences = [text_1, texts[i]]

# Generate embeddings
     embeddings = model.encode(sentences)

# Calculate cosine similarity
     similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
     print("Cosine similarity (BERT): with", database[i], 'is', similarity[0][0])
     d.update({database[i]: similarity[0][0]})

Cosine similarity (BERT): with https://greylock.com/ is 0.48100775
Cosine similarity (BERT): with https://a16z.com/ is 0.51767385
Cosine similarity (BERT): with https://www.sequoiacap.com/ is 0.3396095
Cosine similarity (BERT): with https://www.indexventures.com/ is 0.39028203
Cosine similarity (BERT): with https://www.kleinerperkins.com/ is 0.33101338
Cosine similarity (BERT): with https://lsvp.com/ is 0.11507377
Cosine similarity (BERT): with https://matrix.vc/ is 0.25777662
Cosine similarity (BERT): with https://www.500.co/ is 0.34620687
Cosine similarity (BERT): with https://www.sparkcapital.com/ is 0.29433692
Cosine similarity (BERT): with https://www.nea.com/ is 0.40399498


In [23]:
d_sorted = dict(sorted(d.items(), key=lambda item: item[1]))
d_sorted

{'https://lsvp.com/': 0.11507377,
 'https://matrix.vc/': 0.25777662,
 'https://www.sparkcapital.com/': 0.29433692,
 'https://www.kleinerperkins.com/': 0.33101338,
 'https://www.sequoiacap.com/': 0.3396095,
 'https://www.500.co/': 0.34620687,
 'https://www.indexventures.com/': 0.39028203,
 'https://www.nea.com/': 0.40399498,
 'https://greylock.com/': 0.48100775,
 'https://a16z.com/': 0.51767385}

In [24]:
import itertools

top_3 = dict(list(d_sorted.items())[-1:-4:-1])
top_3

{'https://a16z.com/': 0.51767385,
 'https://greylock.com/': 0.48100775,
 'https://www.nea.com/': 0.40399498}

# Adding the new website's url and it's scraped text into our database.

In [25]:
database.append(my_url)
texts.append(text_1)

In [26]:
database

['https://greylock.com/',
 'https://a16z.com/',
 'https://www.sequoiacap.com/',
 'https://www.indexventures.com/',
 'https://www.kleinerperkins.com/',
 'https://lsvp.com/',
 'https://matrix.vc/',
 'https://www.500.co/',
 'https://www.sparkcapital.com/',
 'https://www.nea.com/',
 'https://deerfield.com/']

In [27]:
len(texts)

11

In [28]:
with open('database', 'wb') as fp:
    pickle.dump(database, fp)

In [30]:
with open ('database', 'rb') as fp:
    database = pickle.load(fp)